# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
# change ... to your location of the Unity environment
env = UnityEnvironment(file_name="...")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

#### Train

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import json
import torch
from collections import deque
from matplotlib import pyplot as plt
from agent import Agent

%matplotlib inline

In [2]:
EPISODE_NUM = 15000
ACTOR_0_PTH = 'checkpoint/ppo_actor_0_ma_demo.pth'
ACTOR_1_PTH = 'checkpoint/ppo_actor_1_ma_demo.pth'
CRITIC_PTH = 'checkpoint/ppo_crtic_ma_demo.pth'
LOG_PATH = 'ma_training_log_demo.json'

In [ ]:
# change ... to your location of the Unity environment
env = UnityEnvironment(file_name="...")

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# get the state size and action size
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

In [ ]:
# Initiate agent
agent = Agent(state_size, action_size)
# Initiate scores collector
scores = []
score_board = deque(maxlen=100)

# Training loop
for episode in range(1, EPISODE_NUM + 1):
    env_info = env.reset(train_mode=True)[brain_name]  
    states = env_info.vector_observations  
    score = 0.
    # reset OUnoise
    agent.noise.reset()
    while True:
        # generate actions and log probabiliies with current actors
        # to interacte with the environment
        actions, log_probs = agent.generate_action(states) 
        env_info = env.step(actions)[brain_name]   
        next_states = env_info.vector_observations                   
        rewards = np.mean(np.array(env_info.rewards))                                  
        dones = np.array(env_info.local_done)                                   
        score += rewards
        # add experiencet of each time step into buffer
        # trigger learning in every UPDATE_PER_STEP time steps
        agent.step(states, actions, log_probs, rewards, dones[0], next_states)
        states = next_states
        if np.any(dones):                                      
            break
    score_board.append(score)
    print(f'episode: {episode}, avarage_score: {score}', end='\r')
    scores.append(score)
    # print average score for last 100 episodes and save the model params periodically.
    if episode % 1000 == 0:
        print(f'episode: {episode}, avarage_score: {np.mean(np.array(score_board))}')
        torch.save(agent.actor_0.state_dict(), ACTOR_0_PTH)
        torch.save(agent.actor_1.state_dict(), ACTOR_1_PTH)
        torch.save(agent.critic_local.state_dict(), CRITIC_PTH)
        with open(LOG_PATH, 'w') as f:
            json.dump(scores, f, indent=4)
            
env.close()

In [ ]:
# Plot the training curve
x = [i for i in range(1, EPISODE_NUM + 1)]
fig, ax = plt.subplots()
ax.plot(x, scores)
ax.set(xlabel='episode', ylabel='score',
       title='Training Curve')
ax.grid()
plt.show()

#### Evaluation

In [7]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque
from matplotlib import pyplot as plt
from happo_every_step import Agent

%matplotlib inline

In [2]:
EPISODE_NUM = 100

In [ ]:
# change ... to your location of the Unity environment
env = UnityEnvironment(file_name="...")

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# get the state size and action size
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

In [ ]:
agent = Agent(state_size, action_size)
# load the saved parameters for actors.
actor_0_params, actor_1_params = torch.load('checkpoint/ppo_actor_0_ma.pth'), torch.load('checkpoint/ppo_actor_1_ma.pth')
agent.actor_0.load_state_dict(actor_0_params)
agent.actor_1.load_state_dict(actor_1_params)

score_board = deque(maxlen=100)

for episode in range(1, EPISODE_NUM + 1):
    env_info = env.reset(train_mode=True)[brain_name]  
    states = env_info.vector_observations  
    score = 0.
    agent.noise.reset()
    while True:
        actions = agent.act(states) 
        env_info = env.step(actions)[brain_name]   
        next_states = env_info.vector_observations                   
        rewards = np.mean(np.array(env_info.rewards))                                  
        dones = np.array(env_info.local_done)   
        # use max score as the target is calculated by taking the maximum over both agents                                
        score += np.max(env_info.rewards)
        states = next_states
        if np.any(dones):                                        
            break
    score_board.append(score)
    print(f'episode: {episode}, score: {score}', end='\r')

print(f'100 episodes average score: {np.mean(np.array(score_board))}')

env.close()

In [ ]:
# Plot the evaluation curve
x = [i for i in range(1, EPISODE_NUM + 1)]
fig, ax = plt.subplots()
ax.plot(x, score_board)
ax.set(xlabel='episode', ylabel='score',
       title='Evaluation Curve')
ax.grid()
plt.show()